# Part 2b: Add Validation

Load and process patient data with BMI calculations.

**Your task:** Add schema and bounds validation to catch data quality issues early.

---

## Load data

In [ ]:
import pandas as pd

# TODO: Define a function validate_schema(df, required_columns) that:
#       - Checks if all required columns are present
#       - Raises ValueError with list of missing columns if any are missing
def validate_schema(df,required_columns):
    required_cols = ["patient_id", "weight_kg", "height_cm", "age"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

# TODO: Define a function validate_bounds(df, bounds_dict) that:
#       - For each column in bounds_dict, check if values are within (min, max)
#       - Use df[col].between(min, max) to find out-of-bounds values
#       - Raises ValueError showing patient_id and value for any out-of-bounds rows
def validate_bounds(df,bounds_dict):
     for col, (min_val, max_val) in bounds_dict.items():
        out_of_bounds = ~df[col].between(min_val, max_val)
        if out_of_bounds.any():
            bad_rows = df[out_of_bounds][["patient_id", col]]
            raise ValueError(f"{col} out of bounds:\n{bad_rows}")
        
df = pd.read_csv("data/patient_intake.csv")

# TODO: Call validate_schema() to check for required columns:
#       ["patient_id", "weight_kg", "height_cm", "age"]
validate_schema(df, ["patient_id", "weight_kg", "height_cm", "age"])

# TODO: Call validate_bounds() with bounds:
#       weight_kg: (30, 250)
#       height_cm: (120, 230)
#       age: (0, 110)
validate_bounds(
    df, 
    {
        "weight_kg": (30, 250),
        "height_cm": (120, 230),
        "age": (0, 110)
    }
)
df.head()

---

## Calculate BMI

In [ ]:
df["height_m"] = df["height_cm"] / 100
df["bmi"] = df["weight_kg"] / (df["height_m"] ** 2)
df["bmi"] = df["bmi"].round(1)

df[["patient_id", "weight_kg", "height_cm", "bmi"]].head()

---

## Categorize BMI

In [ ]:
df["bmi_category"] = pd.cut(
    df["bmi"],
    bins=[0, 18.5, 25, 30, float("inf")],
    labels=["Underweight", "Normal", "Overweight", "Obese"],
    right=False
)

df[["patient_id", "bmi", "bmi_category"]].head()

---

## Summary statistics

In [ ]:
summary = df.groupby("bmi_category")["patient_id"].count()
print("\nBMI category distribution:")
print(summary)

high_risk = df[df["bmi"] > 30]
print(f"\nHigh-risk patients (BMI > 30): {len(high_risk)}")